# Copyright Netherlands eScience Center <br>
** Function     : Regrid fields from ORAS4 to ERA-Interim (Curvilinear to Geographical) ** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2019.05.20 ** <br>
** Last Update  : 2019.05.20 ** <br>
Description     : This notebook aims to regrid fields on curvilinear grid (ORCA02) of ORAS4 to the geographical grid of ERA-Interim. We apply the nearest-neighbour interpolation to avoid un-realistic value in case.<br>
Return Values   : netCDF4 <br>
Caveat          : The interpolation is realized through the package iris, from MET Office. More information about regridding with Iris is available through its official documentation:<br>
https://scitools.org.uk/iris/docs/latest/userguide/interpolation_and_regridding.html <br>

In [1]:
import numpy as np
import scipy as sp
import time as tttt
from netCDF4 import Dataset,num2date
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.ticker as mticker
import iris
import iris.plot as iplt
import cartopy
import cartopy.crs as ccrs

In [2]:
################################   Input zone  #########################################
# specify starting and ending time
# specify data path
# ERAI 3D fields on pressure level
datapath_ERAI = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ERA-Interim/sic_daily'
#datapath_ERAI = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/ERAI/regression'
datapath_ORAS4 = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/warehouse_met'
datapath_ORAS4_mask = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/ORAS4'
# specify output path for figures
output_path = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ORAS4'
########################################################################################

In [4]:
if __name__=="__main__":
    print ('*********************** get the key to the datasets *************************')
#     dataset_ERAI_fields_SIC_SST_SLP = Dataset(os.path.join(datapath_ERAI,
#                                               'surface_ERAI_monthly_regress_1979_2016.nc'))
    dataset_ERAI_fields_SIC_SST_SLP = Dataset(os.path.join(datapath_ERAI, 'era1979',
                                              'pressure_daily_075_diagnostic_1979_10_sic.nc'))
    dataset_ORAS4_OHC = Dataset(os.path.join(datapath_ORAS4,
                                             'oras_model_monthly_1958_2017_ohc.nc'))
    # mask
    dataset_ORAS4_mask = Dataset(os.path.join(datapath_ORAS4_mask, 'mesh_mask.nc'))
    print ('*********************** extract variables *************************')
    SIC = dataset_ERAI_fields_SIC_SST_SLP.variables['siconc'][0,:,:]
    mask_SIC = np.ma.getmask(SIC[:])
    latitude_ERAI = dataset_ERAI_fields_SIC_SST_SLP.variables['latitude'][:]
    longitude_ERAI = dataset_ERAI_fields_SIC_SST_SLP.variables['longitude'][:]
    OHC_300_ORAS4 = dataset_ORAS4_OHC.variables['OHC_300'][20:,:,:,:]/1000 # from 1978 - 2017 unit Peta Joule
    year = dataset_ORAS4_OHC.variables['year'][20:]
    month = dataset_ORAS4_OHC.variables['month'][:]
    gphit_OHC_ORAS4 = dataset_ORAS4_OHC.variables['gphit'][:,:]
    glamt_OHC_ORAS4 = dataset_ORAS4_OHC.variables['glamt'][:,:]    

*********************** get the key to the datasets *************************
*********************** extract variables *************************


In [5]:
    print ('*********************** regridding with iris *************************')
    OHC_regrid = np.zeros((len(year),len(month),len(latitude_ERAI),len(longitude_ERAI)), dtype=float)
    for i in np.arange(len(year)):
        for j in np.arange(len(month)):
            # load the target fields
            OHC = OHC_300_ORAS4[i,j,:,:]
            # basic dimensions for cube in iris
            lat_iris = iris.coords.AuxCoord(gphit_OHC_ORAS4, standard_name='latitude', long_name='latitude',
                                            var_name='lat', units='degrees')
            lon_iris = iris.coords.AuxCoord(glamt_OHC_ORAS4, standard_name='longitude', long_name='longitude',
                                            var_name='lon', units='degrees')
            # assembly the cube
            cube_iris = iris.cube.Cube(OHC, long_name='curvilinear field', var_name='field', 
                                       units='1', aux_coords_and_dims=[(lat_iris, (0,1)), (lon_iris, (0,1))])
            coord_sys = iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS)
            cube_iris.coord('latitude').coord_system = coord_sys
            cube_iris.coord('longitude').coord_system = coord_sys 
            # different methods for interpolation
            # weight interpolation / nearest neighbour interpolation / nearest neighbour interpolation for visualization only
            method = 'neighbour' 
            if method == 'neighbour': # nearest neighbour interpolation
                # more detials about this method
                # https://scitools.org.uk/iris/docs/latest/iris/iris/experimental/regrid.html
                # choose the projection map type
                projection = ccrs.PlateCarree()
                # assemble the target cube
                # create grid_cube for regridding, this is a dummy cube with desired grid
                #lat_grid = np.linspace(-90, 90, 181)
                lat_grid = latitude_ERAI
                # lon_grid = np.linspace(-180, 180, 361)
                lon_grid = longitude_ERAI
                # interpolate_points = [('latitude', np.linspace(-90, 90, 181)),
                #                       ('longitude', np.linspace(-180, 181, 361))]
                lat_aux = iris.coords.DimCoord(lat_grid, standard_name='latitude',
                                               units='degrees_north', coord_system='GeogCS')
                lon_aux = iris.coords.DimCoord(lon_grid, standard_name='longitude',
                                               units='degrees_east', coord_system='GeogCS')
                dummy_data = np.zeros((len(lat_grid), len(lon_grid)))
                cube_tar = iris.cube.Cube(dummy_data,dim_coords_and_dims=[(lat_aux, 0), (lon_aux, 1)])
                # create the coordinate system for the target cube
                cube_tar.coord('latitude').guess_bounds()
                cube_tar.coord('longitude').guess_bounds()
                cube_tar.coord('latitude').coord_system = coord_sys
                cube_tar.coord('longitude').coord_system = coord_sys
                # create a weight matrix for regridding
                weights = np.ones(cube_iris.shape)
                # get regridder from given cubes
                base = iris.analysis.UnstructuredNearest()
                regridder = base.regridder(cube_iris,cube_tar)
                # Transform cube to target projection
                cube_regrid = regridder(cube_iris)
            elif method == 'neighbour_fastview': # nearest neighbour interpolation for a quick visualization only
                # choose the projection map type
                projection = ccrs.PlateCarree()
                # Transform cube to target projection
                # this method only means for a fast visualization. We can not choose target coordinate
                cube_regrid, extent = iris.analysis.cartography.project(cube_iris, projection,
                                                                        nx=len(longitude_ERAI), ny=len(latitude_ERAI))    
            elif method == 'weight':
                # we have to define a weight to make the regridding work
                # create grid_cube for regridding, this is a dummy cube with desired grid
                #lat_grid = np.linspace(-90, 90, 181)
                lat_grid = latitude_ERAI
                # lon_grid = np.linspace(-180, 180, 361)
                lon_grid = longitude_ERAI
                # interpolate_points = [('latitude', np.linspace(-90, 90, 181)),
                #                       ('longitude', np.linspace(-180, 181, 361))]
                lat_aux = iris.coords.DimCoord(lat_grid, standard_name='latitude',
                                               units='degrees_north', coord_system='GeogCS')
                lon_aux = iris.coords.DimCoord(lon_grid, standard_name='longitude',
                                               units='degrees_east', coord_system='GeogCS')
                dummy_data = np.zeros((len(lat_grid), len(lon_grid)))
                aux_cube = iris.cube.Cube(dummy_data,dim_coords_and_dims=[(lat_aux, 0), (lon_aux, 1)])
                # Feed cube with coordinate system
                aux_cube.coord('latitude').guess_bounds()
                aux_cube.coord('longitude').guess_bounds()
                aux_cube.coord('latitude').coord_system = coord_sys
                aux_cube.coord('longitude').coord_system = coord_sys
                # create a weight matrix for regridding
                weights = np.ones(cube_iris.shape)
                # interpolate from ORCA grid to rectilinear grid through bilinear interpolation
                # The method uses point in cell interpolation and then perform the bilinear interpolation
                # based on distance and weight
                cube_regrid = iris.experimental.regrid.regrid_weighted_curvilinear_to_rectilinear(cube_iris,weights,aux_cube)
            else:
                print('This module only supports nearest neighbour interpolation or bilinear interpolation!')
            # save the output fields after regridding
            OHC_regrid[i,j,:,:] = np.ma.masked_array(cube_regrid.data,mask_SIC)
            OHC_regrid[OHC_regrid>1E+10] = 0 # clean up unrealistic data
            OHC_regrid[OHC_regrid<-1E+10] = 0 # clean up unrealistic data
            # set filled value
            np.ma.set_fill_value(OHC_regrid,0)

*********************** regridding with iris *************************


In [7]:
    print ('*********************** output visualization *************************')
    # interpolation complete!!
    print (cube_regrid)
    data_regrid = cube_regrid.data
    y_coord = cube_regrid.coord('latitude').points
    x_coord = cube_regrid.coord('longitude').points
    print(y_coord[:])
    print(latitude_ERAI)
    print(x_coord[:])  
    print(data_regrid)

*********************** output visualization *************************
curvilinear field / (1)             (latitude: 67; longitude: 480)
     Dimension coordinates:
          latitude                           x              -
          longitude                          -              x
[89.5  88.75 88.   87.25 86.5  85.75 85.   84.25 83.5  82.75 82.   81.25
 80.5  79.75 79.   78.25 77.5  76.75 76.   75.25 74.5  73.75 73.   72.25
 71.5  70.75 70.   69.25 68.5  67.75 67.   66.25 65.5  64.75 64.   63.25
 62.5  61.75 61.   60.25 59.5  58.75 58.   57.25 56.5  55.75 55.   54.25
 53.5  52.75 52.   51.25 50.5  49.75 49.   48.25 47.5  46.75 46.   45.25
 44.5  43.75 43.   42.25 41.5  40.75 40.  ]
[89.5  88.75 88.   87.25 86.5  85.75 85.   84.25 83.5  82.75 82.   81.25
 80.5  79.75 79.   78.25 77.5  76.75 76.   75.25 74.5  73.75 73.   72.25
 71.5  70.75 70.   69.25 68.5  67.75 67.   66.25 65.5  64.75 64.   63.25
 62.5  61.75 61.   60.25 59.5  58.75 58.   57.25 56.5  55.75 55.   54.25
 53.5  52

In [9]:
    print ('*********************** make netCDF *************************')
    create_netcdf_point(OHC_regrid, year, month, latitude_ERAI, longitude_ERAI, output_path)

*********************** make netCDF *************************
*******************************************************************
*********************** create netcdf file*************************
*******************************************************************
Start creating netcdf file for OHC from 1978 to 2017.
Create netcdf file successfully


In [8]:
# save output datasets
# we only pack our timeseries from 1979 to 2016
def create_netcdf_point (field, period, week, latitude, longitude, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    print("Start creating netcdf file for OHC from 1978 to 2017.")
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(output_path + os.sep + 'ohc_monthly_oras2erai_1978_2017.nc','w',format = 'NETCDF4')
    # create dimensions for netcdf data
    year_wrap_dim = data_wrap.createDimension('year', len(period))
    week_wrap_dim = data_wrap.createDimension('week', len(week))
    lat_wrap_dim = data_wrap.createDimension('latitude', len(latitude))
    lon_wrap_dim = data_wrap.createDimension('longitude', len(longitude))
    # create coordinate variables for 3-dimensions
    year_wrap_var = data_wrap.createVariable('year',np.int32,('year',))
    week_wrap_var = data_wrap.createVariable('week',np.int32,('week',))
    lat_wrap_var = data_wrap.createVariable('latitude',np.float64,('latitude',))
    lon_wrap_var = data_wrap.createVariable('longitude',np.float64,('longitude',))    
    # create the actual 4-d variable
    field_wrap_var = data_wrap.createVariable('OHC',np.float64,('year','week','latitude','longitude'))

    # global attributes
    data_wrap.description = 'Monthly Ocean Heat Content regridded on rectilinear grid'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lon_wrap_var.units = 'degree_east'
    field_wrap_var.units = 'Peta Joule'
    field_wrap_var.long_name = 'Ocean Heat Content'

    # writing data
    year_wrap_var[:] = period
    week_wrap_var[:] = week
    lat_wrap_var[:] = latitude
    lon_wrap_var[:] = longitude
    field_wrap_var[:] = field

    # close the file
    data_wrap.close()
    print ("Create netcdf file successfully")